In [ ]:
# Treiber für Firefox, Chrome, Edge, etc.
from selenium import webdriver

# Art der Lokalisierung von HTML-Elementen: ID, CSS_SELECTOR, XPATH, ...
from selenium.webdriver.common.by import By               

# Verwendung spezifischer Tasten wie ENTER, ESCAPE, ...
from selenium.webdriver.common.keys import Keys

# Methoden für Dropdown-Menüs
from selenium.webdriver.support.select import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#from selenium.webdriver.firefox.options import Options

import pandas as pd
import re
import time

# Firefox im headless-Modus einrichten
#firefox_options = Options()
#firefox_options.add_argument("--headless")
#driver = webdriver.Firefox(options=firefox_options)

Wir starten einen Browser (hier Firefox) und öffnen eine Webseite.

In [ ]:
# Steam-Bewertungsseite öffnen
driver = webdriver.Firefox()
url = 'https://steamcommunity.com/app/313120/reviews/?browsefilter=toprated&snr=1_5_100010_'
driver.get(url)

Legt den Zeitraum fest, den der Treiber wartet, bis die Elemente erscheinen. Es wird kein Fehler ausgegeben, wenn das Element innerhalb des Zeitraums nicht gefunden wird.

In [3]:
driver.implicitly_wait(10)

#### Cookie Consent Popup

Bevor wir Inhalte der Webseite scrapen können, müssen wir uns mit dem Cookie-Einwilligungs-Popup befassen.

In [ ]:
# Cookies akzeptieren, wenn die Schaltfläche sichtbar ist
# Warte explizit darauf, dass das Element angezeigt wird (bis zu 10 Sekunden)
wait = WebDriverWait(driver, 10)

# Verwendung von XPath um pop-up zu finden
button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Accept All']/..")))
button.click()

In [9]:
# Starte mit der aktuellen Höhe der Seite
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scrolle bis ans Ende der Seite
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)  # Warte, damit die neuen Inhalte geladen werden

    # Berechne die neue Höhe der Seite
    new_height = driver.execute_script("return document.body.scrollHeight")

    # Prüfe, ob neue Inhalte geladen wurden
    if new_height == last_height:
        # Wenn die Seite nicht mehr weiter wächst, ist das Ende erreicht
        break

    last_height = new_height

In [ ]:
# Leere Listen für die gescrapten Daten
comments = []
dates = []
hours_played = []
titles = []

# Finde alle Reviews basierend auf der jeweiligen Klasse
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.apphub_CardTextContent')
posted_dates = driver.find_elements(By.CSS_SELECTOR, 'div.date_posted')
hours = driver.find_elements(By.CSS_SELECTOR, 'div.hours')
review_titles = driver.find_elements(By.CSS_SELECTOR, 'div.title')

# Speichere die Reviews, Titel, Stunden und Datum
for review, date, hour, title in zip(reviews, posted_dates, hours, review_titles):
    # Bereinigen des Kommentartextes, indem alles nach dem Schlüsselwort "EARLY ACCESS REVIEW" extrahiert wird
    clean_comment = re.split(r'EARLY ACCESS REVIEW\n', review.text)[-1].strip()  
    comments.append(clean_comment)
    dates.append(date.text)
    hours_played.append(hour.text)
    titles.append(title.text)

    # Daten in einen DataFrame laden
df = pd.DataFrame({
    'Title': titles,
    'Date Posted': dates,
    'Hours Played': hours_played,
    'Review': comments
})

# DataFrame als CSV-Datei speichern
df.to_csv('steam_reviews.csv', index=False)
print("Daten erfolgreich in steam_reviews.csv gespeichert")

Daten erfolgreich in steam_reviews.csv gespeichert


In [12]:
# Erlaube die vollständige Anzeige von langen Strings in Zellen
pd.set_option('display.max_colwidth', None)

# CSV-Datei einlesen
df = pd.read_csv('steam_reviews.csv')

# Die letzten 5 Zeilen der Datei anzeigen
df.size

84240

In [16]:
df.shape

(21060, 4)

In [17]:
df.head()

Title               Date Posted           Hours Played  \
0  Recommended  Posted: 25 January, 2015     84.1 hrs on record   
1  Recommended  Posted: 7 December, 2019     12.0 hrs on record   
2  Recommended      Posted: 18 May, 2020     63.1 hrs on record   
3  Recommended   Posted: 15 August, 2021     16.2 hrs on record   
4  Recommended  Posted: 4 December, 2020  1,990.9 hrs on record   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [27]:
df['Title'].value_counts()

Title
Recommended        15030
Not Recommended     6030
Name: count, dtype: int64